In [1]:
import pandas as pd
from rank_bm25 import *
import warnings
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import json
from nltk import WordNetLemmatizer
from nltk.stem import PorterStemmer, ISRIStemmer
import re
import string
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
from tkinter import *
import tkinter as tk
import time
import json
import os

# LOAD DATA

In [2]:
#!pip install spacy
path =os.path.abspath('husna.json')
print(path)

C:\Users\Lenovo\Desktop\search engine\husna.json


In [3]:
with open(path,encoding='utf-8') as f:
      data =json.load(f)
        
print(type(data))
data[0]['content']

<class 'list'>


['أكدت أمين عام  وزارة  التربية والتعليم للشؤون المالية والإدارية الدكتورة نجوى القبيلات أن التوسع في دوام الطلبة في مدارس وزارة التربية على نظام الفترتين جاء بسبب الاكتظاظ وزيادة أعداد الطلبة الذين انتقلوا إلى المدارس الحكومية والنقص في الأبنية المدرسية.',
 'وأشارت قبيلات إلى تحويل 42 مدرسة إلى نظام الفترتين, واستئجار 15 مدرسة خلال هذا العام، علما بأن عدد المدارس الحكومية التي تعمل بنظام الفترتين 820 مدرسة.',
 'وبيّنت قبيلات لـ  حسنى ، حاجة وزارة التربية إلى بناء 600 بناء مدرسي جديد خلال السنوات الـ 10 المقبلة وفق الاستراتيجية التي وضعتها وزارة الـتربية.',
 'وفيما يتعلق بتبرع أحد المواطنين ببناء مدرسة في منطقة ارحاب بمحافظة  المفرق  وما زال ينتظر منذ سنة ونصف تسليمها إلى وزارة الـتربية، أكدت قبيلات أن الوزارة ستتسلم المدرسة ولكن هذا الأمر يحتاج إلى دراسة وإجراءات، منوّهة أن الوزارة ومن خلال مديرية الأبنية طلبت من الجمعية العلمية التأكد من سلامة البناء.',
 'وأكدت قبيلات أن الوزارة أنجزت مسحا وطنيا لمعرفة حاجتها من المدارس وتوزيعها في مختلف المحافظات، وبالنسبة للمتبرعين سواء تبرعهم بأبن

# Preproc Data

In [5]:

def clean_text(text):
#will replace the html characters with " "
    text=re.sub('<.*?>', ' ', text)  
    #To remove the punctuations
    text = text.translate(str.maketrans(' ',' ',string.punctuation))
    #will consider only alphabets and numerics
    text = re.sub('[^a-zA-Z]',' ',text)  
    #will replace newline with space
    text = re.sub("\n"," ",text)
    #will convert to lower case
    text = text.lower()
    # will split and join the words
    text=' '.join(text.split())
    return text

stop_words = set(stopwords.words('arabic'))
def removal_stopwords(text):
    word_tokens = word_tokenize(str(text))  
    filtered_sentence = []
    a_row=""
    for a_word in word_tokens:
        if a_word not in stop_words:
            filtered_sentence.append(a_word)
            a_row = " ".join(filtered_sentence)
    return a_row


stemer= ISRIStemmer()
def stemming(text):
    word_tokens = word_tokenize(str(text))  
    a_array=[]
    a_string = ""
    for a_word in word_tokens:
   
       a_stem = stemer.stem(a_word)
   
       a_array.append(a_stem)
    
       a_string = " ".join(a_array)
    return a_string

In [6]:
len(data)

6366

In [7]:
# 1.Tokenization is the process of breaking up the paragraph into smaller units such as sentences or words. Each unit in
#then considered as an individual token. 


print ("####################################")
for i in range (len(data)):
        data [i]['content_new'] = removal_stopwords(data[i]['content'])
data[2]['content_new']



####################################


"[ 'يشهد العالم أزمة طاقة عالمية حقيقية التاريخ.\\xa0مع تنامي المخاوف انقطاع إمدادات الغاز الروسي ، وكما حذرت وكالة الطاقة الدولية لعدة أشهر ، الوضع محفوف بالمخاطر بشكل خاص أوروبا، تقع بؤرة اضطراب سوق الطاقة.\\xa0ما سيخلق حالة القلق بشكل خاص خلال الأشهر المقبلة . ' , 'وكان دور روسيا واضحًا البداية.\\xa0ففي 2021 - أشهر الغزو الروسي لأوكرانيا -\\xa0أشارت وكالة الطاقة الدولية إلى\\xa0أن موسكو تمنع وصول كمية كبيرة الغاز أوروبا . خلق حالة إرباك مصطنعة الأسواق شهدت ارتفاعا بالأسعار تصاعد التوترات حول أوكرانيا . ' , 'بعد غزو روسيا لأوكرانيا 24 الماضي ، أوروبا مكان آخر يمكن يكون لديه أوهام حول المخاطر المحيطة بإمدادات الطاقة الروسية.\\xa0بعد أسبوع فقط بدء الغزو ، أصدرت وكالة الطاقة الدولية خطة\\xa0من 10 نقاط لتقليل اعتماد الاتحاد الأوروبي الغاز الروسي الطبيعي\\xa0، حددت خلالها الإجراءات العملية يمكن تتخذها أوروبا.\\xa0وشددت الخطة نقاط رئيسة تضمنت ' , 'ويرى خبراء ومختصون أوروبا مجبرة العمل حالة دائمة عدم اليقين بشأن إمدادات الغاز الروسي، وأنه يمكن استبعاد احتمالية قطع إمدادات الغاز الروسي بشكل 

In [8]:
#3 - Stemming is the process of reduction of a word into its root or stem word. 
#The word affixes are removed leaving behind only the root form or lemma.
for i in range (len(data)):
        data [i]['content_new'] = stemming(data[i]['content_new'])
data[2]['content_new']


"[ 'يشهد علم ازم طقة علم حقق تاريخ.\\xa0مع نمي خاف قطع مدد غاز روس ، وكم حذر وكل طقة دول لعد شهر ، وضع حفف خطر شكل خاص روبا، تقع بؤر ضطراب سوق طاقة.\\xa0 خلق حلة قلق شكل خاص خلل شهر قبل . ' , 'وك دور روس وضح بداية.\\xa0ففي 2021 - شهر غزو روس أوكر -\\xa0أشارت وكل طقة دول الى\\xa0أن وسكو منع وصل كمة كبر غاز ورب . خلق حلة ربك صطنع سوق شهد رفع سعر صعد وتر حول وكر . ' , 'بعد غزو روس أوكر 24 اضي ، ورب كان اخر يمكن يكون لده وهم حول خطر حيط إمداد طقة روسية.\\xa0بعد سبع فقط بدء غزو ، صدر وكل طقة دول خطة\\xa0من 10 نقط قلل عمد تحد ورب غاز روس طبيعي\\xa0، حدد خلل جرء عمل يمكن تخذ اوروبا.\\xa0وشددت خطة نقط رئس تضم ' , 'ويرى خبراء ختص ورب جبر عمل حلة دئم عدم يقن بشأ مدد غاز روسي، وأن يمكن بعد احتمالية قطع مدد غاز روس شكل كامل، يضف خبراء انه فضل كثر تخذ خطو عدد فصل شتء بدل رفه ئات لاي ورب قصد ورب رحم طقس، اسأ ذلك، عطء نفذ ضفي ضرر رئس روس فلاديمير بوت . ' , 'فف اضي وقف دفق عبر بوب ورد ريم، كبر خط ابب غاز روس وأوروبا، علل روس وقف بأن ضمن خطط صين كانت قرر مسبقا.\\xa0وكانت روس خفض فعل دفق قدم عبر ورد ترم

In [9]:
print(data[0]['content'])
print("###################")
data[0]['content_new']

['أكدت أمين عام  وزارة  التربية والتعليم للشؤون المالية والإدارية الدكتورة نجوى القبيلات أن التوسع في دوام الطلبة في مدارس وزارة التربية على نظام الفترتين جاء بسبب الاكتظاظ وزيادة أعداد الطلبة الذين انتقلوا إلى المدارس الحكومية والنقص في الأبنية المدرسية.', 'وأشارت قبيلات إلى تحويل 42 مدرسة إلى نظام الفترتين, واستئجار 15 مدرسة خلال هذا العام، علما بأن عدد المدارس الحكومية التي تعمل بنظام الفترتين 820 مدرسة.', 'وبيّنت قبيلات لـ  حسنى ، حاجة وزارة التربية إلى بناء 600 بناء مدرسي جديد خلال السنوات الـ 10 المقبلة وفق الاستراتيجية التي وضعتها وزارة الـتربية.', 'وفيما يتعلق بتبرع أحد المواطنين ببناء مدرسة في منطقة ارحاب بمحافظة  المفرق  وما زال ينتظر منذ سنة ونصف تسليمها إلى وزارة الـتربية، أكدت قبيلات أن الوزارة ستتسلم المدرسة ولكن هذا الأمر يحتاج إلى دراسة وإجراءات، منوّهة أن الوزارة ومن خلال مديرية الأبنية طلبت من الجمعية العلمية التأكد من سلامة البناء.', 'وأكدت قبيلات أن الوزارة أنجزت مسحا وطنيا لمعرفة حاجتها من المدارس وتوزيعها في مختلف المحافظات، وبالنسبة للمتبرعين سواء تبرعهم بأبنية م

"[ 'أكد امن عام وزر ربي علم شؤن الة دري دكتور نجى قبل وسع دوم طلب درس وزر ربي نظم فتر جاء سبب كظظ زيد عدد طلب نقل درس حكم نقص بني درس . ' , 'وأشار قبل حول 42 درس نظم فتر , واستئجار 15 درس خلل عم، علم بأن عدد درس حكم عمل نظم فتر 820 درس . ' , 'وب قبل لـ حسنى ، حجة وزر ربي بنء 600 بنء درس جدد خلل سنو الـ 10 قبل وفق استراتيجية وضع وزر ـترب . ' , 'وف علق برع وطن بنء درس نطق رحب حفظ فرق زال نظر سنة نصف سلم وزر ـتربية، اكد قبل وزر سلم درس امر حاج درس وإجراءات، نوه وزر خلل دير بني طلب جمع علم أكد سلم بنء . ' , 'وأكد قبل وزر جزت سحا وطن عرف حجت درس وزع خلف محافظات، وبالنسبة تبرع سوء برع بأب درس رض، فقد طلب وزر تبرع نسق سبق عها حدد كان رحل بنء حدد حجة . ' ]"

# creat corpus 

In [19]:
a=[]
for i in range(len(data)):
    a.append(data[i]["content_new" ])

In [44]:
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
#converts the document into lower-case tokens lust,
#ignoring the ones that are too short or too long.
doc_tokenized =(simple_preprocess(doc) for doc in a)
print(doc_tokenized)
dictionary = Dictionary()


corpus = [dictionary.doc2bow(desc, allow_update=True) for desc in doc_tokenized]


#print(word_frequencies[3:10])

<generator object <genexpr> at 0x000002B35AEF3BA0>


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [45]:
%time tfidf_model = gensim.models.TfidfModel(corpus, id2word=dictionary)
%time lsi_model = gensim.models.LsiModel(tfidf_model[corpus], id2word=dictionary,num_topics=100)


CPU times: total: 234 ms
Wall time: 235 ms
CPU times: total: 7.55 s
Wall time: 5.72 s


In [22]:
%time gensim.corpora.MmCorpus.serialize('tfidf_model_mm', tfidf_model[corpus])
%time gensim.corpora.MmCorpus.serialize('lsi_model_mm',lsi_model[tfidf_model[corpus]])

CPU times: total: 5.22 s
Wall time: 5.27 s
CPU times: total: 4.77 s
Wall time: 4.8 s


In [23]:
#Load the indexed corpus
tfidf_corpus = gensim.corpora.MmCorpus('tfidf_model_mm')
lsi_corpus = gensim.corpora.MmCorpus('lsi_model_mm')

print(tfidf_corpus)
print(lsi_corpus)

MmCorpus(6366 documents, 34636 features, 662746 non-zero entries)
MmCorpus(6366 documents, 100 features, 499800 non-zero entries)


In [24]:
from gensim.similarities import MatrixSimilarity

%time index = MatrixSimilarity(lsi_corpus, num_features = lsi_corpus.num_terms)
print (index)

CPU times: total: 906 ms
Wall time: 916 ms
MatrixSimilarity<6366 docs, 100 features>


# Search engine function

In [42]:
from operator import itemgetter
def search_similar(search_term):
    print('search : ',search_term)
    startTime = time.time()
    search_term= simple_preprocess(search_term)
    query_bow = dictionary.doc2bow( search_term)
    query_tfidf = tfidf_model[query_bow]
    query_lsi = lsi_model[query_tfidf]
    index.num_best =6
    _list = index[query_lsi]
     
    _list.sort(key=itemgetter(1), reverse=True)
    names = []

    for j, d in enumerate(_list):
        names.append (
            {
                'Relevance': round((d[1] * 100),2),
                'Title':data[d[0]]['title'],
                'URL' :data[d[0]]['url']
            }

        )
        if j == (index.num_best-1):
            break
    executionTime = (time.time() - startTime)
    print('Execution time in seconds: ' + str(executionTime))
    return pd.DataFrame(names, columns=['Relevance','Title','URL'])  

In [43]:
%time search_similar(' الخصاونة  ')


search :   الخصاونة  
Execution time in seconds: 0.0019958019256591797
CPU times: total: 0 ns
Wall time: 2.99 ms


,Relevance,Title,URL
0,59.32,الخصاونة للنواب: ما جرى لم يكن انقلاباً وإنما ...,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...
1,52.88,رئيس الوزراء يوجه لإجراء تحقيق فوري في حادثة ا...,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...
2,49.92,الحراسيس: ما جرى مع النائب أبو حسان اختلاف بوج...,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D8%...
3,48.98,معركة كراسي في اليوم الأول للدورة الاستثنائية ...,https://husna.fm/%D8%AD%D8%B3%D9%86%D9%89/%D9%...
4,48.97,الخصاونة يبحث مع نظيره العراقي تعزيز التعاون ا...,https://husna.fm/%D8%B9%D8%B1%D8%A8%D9%8A-%D9%...
5,47.56,قبول استقالة مدير عام الجمارك عبدالمجيد الرحامنة,https://husna.fm/%D9%85%D8%AD%D9%84%D9%8A/%D9%...


# GUI SEARCH

In [ ]:
struct=Tk()
struct.geometry("354x350") #Defining Size of GUI box
struct.title('Search Engine') 
label=Label(struct,text="Search Engine",bg="teal",fg="white",font=("Times",20,"bold"))
label.pack(side=TOP) 
struct.config(background="teal")



def search():
    
    inp=inputtxt.get('1.0','end')
    out=search_similar(inp)
    print(out)
    print("##############################################################")

  
label= Label(struct,text="Enter here to search",bg="teal",fg="white",font=("Times",15,"bold"))
label.place(x=80,y=100)
inputtxt = tk.Text(struct,
                   height = 2,
                   width =30,bd=2)
inputtxt.place(x=80,y=140)

button=Button(struct,text="Search",font=("Times",10,"bold"),width=30,bd=2,command=search)
button.place(x=80,y=180)
struct.mainloop()

